In [3]:
import numpy as np
import DeepLearning
import seaborn as sns

In [2]:
import tensorflow as tf

mnist_d = tf.keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist_d.load_data()
def argmax(xs):
    return max( range(len(xs)), key=lambda i: xs[i] )

In [3]:
train_images = train_images/225
test_images = test_images/225

train_images = train_images.reshape(train_images.shape[0], -1)
test_images = test_images.reshape(test_images.shape[0], -1)

In [4]:
def oneHotEncode(i, num_labels):
    return np.array( [ 1 if i == j else 0 for j in range(num_labels)] )

train_labels = np.array([oneHotEncode(label, 10) for label in train_labels])
test_labels = np.array([oneHotEncode(label, 10) for label in test_labels])

In [5]:
def loop(model : DeepLearning.Sequential, trainVars, trainTargets, loss: DeepLearning.Loss, optimizer : DeepLearning.Optimizer = None):
    correct = 0

    for i in range(len(trainVars)):
        predicted = model.forward(trainVars[i])
        if argmax(predicted) == argmax(trainTargets[i]):
            correct += 1

        #If is training
        if optimizer is not None:
            gradient = loss.gradient(predicted, trainTargets[i])
            model.backward(gradient)
            optimizer.step(model)

    return correct/len(trainVars)


In [6]:
dropout1 = DeepLearning.Dropout(0.1)
dropout2 = DeepLearning.Dropout(0.1)
predModel = DeepLearning.Sequential([
    DeepLearning.Linear(784, 30, name='Linear1'),
    dropout1,
    DeepLearning.TANH(),
    DeepLearning.Linear(30, 10, name='Linear2'),
    dropout2,
    DeepLearning.TANH(),
    DeepLearning.Linear(10,10, name='Linear3')

])

loss = DeepLearning.SoftMaxCrossEntropy()
optimizer = DeepLearning.Momentum(learning_rate=0.01, mo=0.99)
dropout1.train = dropout2.train = True
loop(predModel, train_images, train_labels, loss, optimizer)

0.8744333333333333

In [8]:
dropout1.train = dropout2.train = False

accuracy = loop(predModel, test_images, test_labels, loss=loss, optimizer=None)
print(accuracy)

0.922


In [11]:
import pickle

fileHandler = open("MNIST_Prediciton_Model", 'wb')
pickle.dump(predModel, fileHandler)
fileHandler.close()
with open("MNIST_Prediciton_Model", 'rb') as f:
    loadedModel = pickle.load(f)